In [1]:
!pip install simple_salesforce

In [2]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
# import sys 
# sys.path.append('../../../../')
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_gwsis_dbname, remote_gwsis_dbuser, remote_gwsis_dbpwd

In [4]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [5]:
engine = create_engine(f"mysql://{remote_gwsis_dbuser}:{remote_gwsis_dbpwd}@{remote_db_endpoint}:{remote_db_port}/{remote_gwsis_dbname}")
conn = engine.connect()

In [6]:
border_data = pd.read_sql("SELECT * FROM border_data", conn)
border_data.head()

,Port Name,State,Port Code,Country,Date,Measure,Value,Latitude,Longitude
0,Calexico East,California,2507,Mexico,3/1/2019,Trucks,34447,-115.48433,32.67524
1,Van Buren,Maine,108,Canada,3/1/2019,Rail Containers Full,428,-67.94271,47.16207
2,Otay Mesa,California,2506,Mexico,3/1/2019,Trucks,81217,-117.05333,32.57333
3,Nogales,Arizona,2604,Mexico,3/1/2019,Trains,62,-110.93361,31.34028
4,Trout River,New York,715,Canada,3/1/2019,Personal Vehicle Passengers,16377,-73.44253,44.99001


In [7]:
border_load = border_data.to_dict('records')
border_load

[{'Port Name': 'Calexico East',
  'State': 'California',
  'Port Code': 2507,
  'Country': 'Mexico ',
  'Date': '3/1/2019',
  'Measure': 'Trucks',
  'Value': 34447,
  'Latitude': -115.48433,
  'Longitude': 32.67524},
 {'Port Name': 'Van Buren',
  'State': 'Maine',
  'Port Code': 108,
  'Country': 'Canada ',
  'Date': '3/1/2019',
  'Measure': 'Rail Containers Full',
  'Value': 428,
  'Latitude': -67.94271,
  'Longitude': 47.16207},
 {'Port Name': 'Otay Mesa',
  'State': 'California',
  'Port Code': 2506,
  'Country': 'Mexico ',
  'Date': '3/1/2019',
  'Measure': 'Trucks',
  'Value': 81217,
  'Latitude': -117.05333,
  'Longitude': 32.57333},
 {'Port Name': 'Nogales',
  'State': 'Arizona',
  'Port Code': 2604,
  'Country': 'Mexico ',
  'Date': '3/1/2019',
  'Measure': 'Trains',
  'Value': 62,
  'Latitude': -110.93361,
  'Longitude': 31.34028},
 {'Port Name': 'Trout River',
  'State': 'New York',
  'Port Code': 715,
  'Country': 'Canada ',
  'Date': '3/1/2019',
  'Measure': 'Personal Vehic

In [ ]:
for index, row in border_data.iterrows():

    record = {
        'Port Name': row['Port Name'],
        'State': row['State'],
        'Port Code': row['Port Code'],
        'Border': row['Border'],
        'Date': row['Date'],
        'Measure': row['Measure'],
        'Value': row['Value'],
        'Location': row['Location']
    }
    
    try:
        sf.GWU_DB.create(record)
    except Exception as e:
        print(e)

In [10]:
try:
    sf.bulk.GWU_DB__c.insert(border_load)
except Exception as e:
    print(e)

Malformed request https://na174.salesforce.com/services/async/38.0/job/7506g000000wilsAAA/batch. Response content: {'exceptionCode': 'ClientInputError', 'exceptionMessage': 'Failed to read request. Exceeded max size limit of 10000000'}
